In [1]:
import os 
os.getcwd()

'/home/tranhuy/Desktop/Project/cantho-public-services/backend'

In [2]:
from langchain.docstore.document import Document
from langchain_core.prompts import PromptTemplate
from rag.modules.app_vars import AppVariables
import re

/home/tranhuy/Desktop/Project/cantho-public-services/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/tranhuy/Desktop/Project/cantho-public-services/venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/tranhuy/Desktop/Project/cantho-public-services/venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/tranhuy/Desktop/Project/cantho-public-se

In [29]:
template = """Bạn là một chatbot hỗ trợ sinh các câu hỏi đồng nghĩa dựa vào câu hỏi gốc về thủ tục hành chính. Hãy tuân thủ các nguyên tắc sau khi tạo ra HAI câu hỏi đồng nghĩa:
    - Hai câu hỏi phải đồng nghĩa và sử dụng MỘT trong bốn các cụm từ sau: 'trình tự các bước', 'cách thức thực hiện', 'hồ sơ cần', 'cơ quan thực hiện', 'điều kiện thực hiện' .
    - Với những câu hỏi chung chung, hãy ưu tiên hỏi một câu về về 'trình tự các bước' .

    Hai câu hỏi phải được phân cách bằng ký tự '\n' . KHÔNG thêm bất kỳ ký tự nào khác ngoài hai câu hỏi được bạn tạo ra.
    Câu hỏi gốc : {question}
"""

template2 = """Bạn là một chatbot hỗ trợ sinh làm rõ câu hỏi của người dùng về thủ tục hành chính. Dựa vào đoạn hội thoại dưới đây, hãy viết lại câu hỏi của người dùng ngắn gọn chứa tên thủ tục hành chính nếu được.
    KHÔNG thêm bất kỳ ký tự nào khác ngoài câu hỏi được bạn tạo ra.

    ------
    Đoạn hội thoại: {dialogue}
    
    ------
    Câu hỏi từ người dùng : {question}

    ------
    Câu hỏi viết lại: 
"""

prompt = PromptTemplate.from_template(template)

llm_chain = prompt | AppVariables.llm
rewrite_chain = PromptTemplate.from_template(template2) | AppVariables.llm

def rewrite_question(question, messages=[]):
    dialogue = "\n"
    for message in messages:
        dialogue += f"{message['role']}: {message['content']}\n"
    
    completion = rewrite_chain.invoke({"question": question, "dialogue": dialogue})
    completion = process_question(completion)
    return completion

def reciprocal_rank_fusion(search_results_dict, k=25):
    fused_scores = {}        
    for query, doc_scores in search_results_dict.items():
        for rank, (doc, score) in enumerate(sorted(doc_scores.items(), key=lambda x: x[1]['score'], reverse=True)):
            if doc not in fused_scores:
                fused_scores[doc] = {
                    "rank": 0,
                    "metadata": score["metadata"]
                }
            fused_scores[doc]["rank"] += 1 / (rank + k)

    reranked_results = sorted(fused_scores.items(), key=lambda x: x[1]['rank'], reverse=True)
    reranked_results = [Document(page_content=doc[0], metadata=doc[1]["metadata"]) for doc in reranked_results]
    return reranked_results

def process_question(question):
    question = question.strip()
    question = question.split(":")[-1].strip()
    question = re.sub(r'^[^a-zA-Z]*', '', question)
    question = re.sub(r'\W+$', '', question)
    return question

def generate_questions(question):
    completion = llm_chain.invoke(question)
    questions = completion.split("\n")

    processed_questions = [ question ]
    for generated_question in questions:
        generated_question = process_question(generated_question)
        if generated_question != "":
            processed_questions.append(generated_question)
    processed_questions = processed_questions[0:min(3, len(processed_questions))]
    return processed_questions

In [30]:
from models import Message, Conversation

messages = [
    {
        "role": "user",
        "content": "muốn làm thủ tục kết hôn cần làm gì"
    },
    {
        "role": "bot",
        "content": "Để đăng ký kết hôn, bạn cần làm theo các bước sau:\n\n1. Nộp hồ sơ tại Ủy ban nhân dân cấp xã có thẩm quyền.\n2. Người tiếp nhận sẽ kiểm tra hồ sơ và đối chiếu thông tin.\n3. Nếu hồ sơ đầy đủ, hợp lệ, người tiếp nhận sẽ viết giấy tiếp nhận.\n4. Hồ sơ sau khi đã được hướng dẫn mà không được bổ sung đầy đủ sẽ bị từ chối.\n5. Trong thời hạn 05 ngày làm việc, công chức tư pháp - hộ tịch sẽ kiểm tra, xác minh hồ sơ.\n6. Nếu việc đăng ký lại kết hôn không phải tại nơi đã đăng ký trước đó, sẽ có bước kiểm tra và xác minh tại nơi đăng ký trước đó.\n7. Sau khi hồ sơ được xác minh, nếu đủ điều kiện, sẽ được giải quyết và cấp Giấy chứng nhận kết hôn.\n\nĐể biết thông tin chi tiết về các giấy tờ cần nộp và thủ tục cụ thể, bạn có thể tham khảo thông tin trong các đoạn văn bản mà mình đã cung cấp .\n"
    }
]
questions = rewrite_question("cần giấy tờ gì", messages=messages)
questions

'Cần những giấy tờ gì để đăng ký kết hôn'

In [32]:
conversation = Conversation.objects(id="663ae632e491cf3d60ea9e5d").first()

messages = Message.objects(conversation=conversation).order_by('created_at').limit(4)
history = []
for message in messages:
    history.append({
        "role": message.role,
        "content": message.message
    })
history

[{'role': 'user', 'content': 'cần làm gì để đăng ký kết hôn'},
 {'role': 'bot',
  'content': 'Để đăng ký kết hôn, bạn cần làm theo các bước sau:\n\n1. Nộp hồ sơ tại Ủy ban nhân dân cấp xã có thẩm quyền.\n2. Người tiếp nhận sẽ kiểm tra hồ sơ và đối chiếu thông tin.\n3. Nếu hồ sơ đầy đủ, hợp lệ, người tiếp nhận sẽ viết giấy tiếp nhận.\n4. Hồ sơ sau khi đã được hướng dẫn mà không được bổ sung đầy đủ sẽ bị từ chối.\n5. Trong thời hạn 05 ngày làm việc, công chức tư pháp - hộ tịch sẽ kiểm tra, xác minh hồ sơ.\n6. Nếu việc đăng ký lại kết hôn không phải tại nơi đã đăng ký trước đó, sẽ có bước kiểm tra và xác minh tại nơi đăng ký trước đó.\n7. Sau khi hồ sơ được xác minh, nếu đủ điều kiện, sẽ được giải quyết và cấp Giấy chứng nhận kết hôn.\n\nĐể biết thông tin chi tiết về các giấy tờ cần nộp và thủ tục cụ thể, bạn có thể tham khảo thông tin trong các đoạn văn bản mà mình đã cung cấp.'}]